In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/apex-codes/entity_sum

/content/drive/My Drive/Colab Notebooks/apex-codes/entity_sum


In [ ]:
!pip3 install Bio

     |████████████████████████████████| 271 kB 5.2 MB/s 
     |████████████████████████████████| 2.3 MB 40.4 MB/s 


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Import Python Library

In [ ]:
import io
from Bio import Entrez, Medline
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import re
import collections
import pandas as pd
import numpy as np
from tqdm import tqdm
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,make_scorer,precision_score,recall_score,roc_auc_score,f1_score
from sklearn.model_selection import GridSearchCV
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import urllib.request, urllib.error, urllib.parse
import json
import os
from pprint import pprint
import nltk
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
from bs4 import BeautifulSoup
from rouge import Rouge

import warnings
warnings.simplefilter("ignore", UserWarning)


## Data Collection & Pre-processing

## Parse medline 

In [ ]:
def search_medline(query):
  Entrez.email = "aman.fanta@gmail.com"
  search = Entrez.esearch(db='pubmed', term=query, retmax = 1000, usehistory='y', reldate=90)
  handle = Entrez.read(search)

  try:
      return handle
  except Exception as e:
      raise IOError(str(e))
  finally:
      search.close()

In [ ]:
def fetch_rec(rec_id, entrez_handle):
    fetch_handle = Entrez.efetch(db='pubmed', id = rec_id, rettype='Medline', retmode='text',
                                 webenv=entrez_handle['WebEnv'],query_key=entrez_handle['QueryKey'], reldate=90)
    rec = fetch_handle.read()
    
    return rec

In [ ]:
def _get_search_query_terms(LEXICON_DIR_PATH, icd_chapter_name):
  with open(f"{LEXICON_DIR_PATH}/{icd_chapter_name}.csv", 'r') as fp:
    disease_list = fp.read().splitlines()
    disease_list = disease_list[:500]   # the first 500 query terms

    query = ""

    # For complete string of query terms Bio Entrez parser expects. "or" is used as the delimiter between two query terms
    for disease in disease_list:
      query = (query) + " or " + (disease)

  fp.close()

  return query


In [ ]:
def _query_parse_medline_abstracts(icd_chapter_name, complete_query_terms) :
  count = 0
  fau = ""
  mh = ""
  j = 0
  rec_handler = search_medline(complete_query_terms)
  count = int(rec_handler["Count"])
  batch_size = 200
  
  df = pd.DataFrame(columns = ['PMID', 'Title', 'Abstract', 'MeSH', 'PubType', 'PubDate', 'Journal', 'DOI', 'PMC'])
          
  for recId in rec_handler['IdList']:

    if j % 100 == 0 and j != 0:
        print(j)
    j += 1

    fau = ""
    mh = ""
    rec = fetch_rec(recId, rec_handler)
    r = io.StringIO(rec)
    medline_rec = Medline.read(r)

    if 'PMID' in medline_rec:
        df.loc[j, 'PMID'] = medline_rec['PMID']

    if 'TI' in medline_rec:
        df.loc[j, 'Title'] = medline_rec['TI']
    else:
        df.loc[j, 'Title'] = 'null'

    if 'AB' in medline_rec:
        df.loc[j, 'Abstract'] = medline_rec['AB']
    else:
        df.loc[j, 'Abstract'] = 'null'

    if 'PT' in medline_rec:
        df.loc[j, 'PubType'] = medline_rec['PT']
    else:
        df.loc[j, 'PubType'] = 'null'

    if 'FAU' in medline_rec:
        for y in medline_rec['FAU']:
            fau = fau + y + ', '
        df.loc[j, 'FAU'] = fau
    else:
        df.loc[j, 'FAU'] = 'null'

    if 'MH' in medline_rec:
        for x in medline_rec['MH']:
            mh = mh + x + ', '
        df.loc[j, 'MeSH'] = mh
    else:
        df.loc[j, 'MeSH'] = 'null'

    if 'DP' in medline_rec:
        df.loc[j, 'PubDate'] = medline_rec['DP']
    else:
        df.loc[j, 'PubDate'] = 'null'

    if 'JT' in medline_rec:
        df.loc[j, 'Journal'] = medline_rec['JT']
    else:
        df.loc[j, 'Journal'] = 'null'

    if 'AID' in medline_rec:
        df.loc[j, 'DOI'] = medline_rec['AID']
    else:
        df.loc[j, 'DOI'] = 'null'

    if 'PMC' in medline_rec:
        df.loc[j, 'PMC'] = medline_rec['PMC']
    else:
        df.loc[j, 'PMC'] = 'null'

    mh = ""
    fau = ""

  return df

In [ ]:
def main():
  LEXICON_DIR_PATH = "SNOMED_CT_and_DataMed_diseases"   # path to lexicons of ICD-11 diseases - Fixed
  icd_chapter_name = "neoplasms"   # changes with each name of ICD 11 chapter
  OUTPUT_DIR_PATH = "ICD_11_Final_Corpus"

  os.makedirs(f'{OUTPUT_DIR_PATH}', exist_ok=True)

  # call to the query_string generator method defined above
  complete_query_terms = _get_search_query_terms(LEXICON_DIR_PATH, icd_chapter_name)   # returns a complete string representation of query terms

  df_complete = _query_parse_medline_abstracts(icd_chapter_name, complete_query_terms)   # Call to method to query pubmed for abstracts

  df_complete.to_excel(f'{OUTPUT_DIR_PATH}/{icd_chapter_name}_Medline.xlsx')   # write the final corpus to the output directory

In [ ]:
if __name__ == "__main__":
  main()

100
200
300
400
500
600
700
800
900
